In [24]:
#PURPOSE: This code will import SPM estimates (compiled using STATA from the CPS-ASEC and the ACS), run a#maximum entropy model to estimate the error of PUMA-level SPM rates, and export the results for each state.#The maximum entropy model will use the state-level SPM rates from the (relatively) more reliable CPS-ASEC to#constrain the PUMA-level SPM rates from the ACS.#ATTRIBUTIONS: Help with the structure of the code was provided by Elissa Cohen.import cvxpy as cp
import numpy as np
import pandas as pd 
import sys 

In [25]:
# Define any functions that will be used later in the code. 
def column(matrix, i):
    return [row[i] for row in matrix]

In [37]:
# Import data: 
data = pd.read_excel(r"C:\Users\Danielle Wilson\Dropbox\SPM_ASEC_ACS_project\acs_cps_pooled_spm_2016_18.xlsx", header=0)

# Create list of unique gestfips values:
unique_gf_set = set(data['state_gestfips']) # Values in set 
gestfips = list(unique_gf_set) # Turn set into list 


## BEGIN LOOP OVER STATES **********************************************
for s in gestfips: 
    
    #Only keep one state: 
    # Data observations only begin at the first cell (0) for AL. Otherwise, they start at whatever row number the first  
    # state record is listed in the XLSX doc. For example, if you keep data with only "state_gestfups == 2", the first line 
    # observation will be record (i.e. cell) 34, not zero. 

    data2= data[data.state_gestfips == s] 
    #print(data2)

    # "Grab" indicies from imported data. 
    index_list = data2.index.tolist()
    # Store the value of the first index 
    first_index = index_list[0]
    
    # "Grab" and prepare data for entropy maximization problem. Data will be saved/stored as numbers or vectors. 
    ##Pumas:
    puma = data2['puma']
    num_pumas = len(puma) #Number of PUMAs in state. 
        #print(num_pumas)

    #CPS data: 
    ##CPS State SPM Rate (Number): 
    r_s_cps = data2['cps_w_spmu_poor_ms'] 
    r_s_cps_num = r_s_cps[first_index] #Only need the first element because this value will be the same for all PUMAs. 
        #print(r_s_cps_num)  

    #ACS data: 
    ##ACS PUMA SPM Rate (List):
    r_sp_acs = data2['w_spmu_poor_ms_p'] #Saves as smaller dataframe. 
    ###Transfrom rate into vector (rather than dataframe). 
    nr_sp_acs = np.matrix(np.reshape(r_sp_acs.to_numpy(), (num_pumas,1)))

    ##ACS PUMA Weighted Population Size (List):
    n_sp_acs = data2['w_count_Ns_p']
    ###Transfrom population size into vector (rather than dataframe).
    nn_sp_acs = np.matrix(np.reshape(n_sp_acs.to_numpy(), (num_pumas,1)))

    ##ACS State Weighted Population Size (Number): 
    n_s_acs = data2['w_count_Ns']
    n_s_acs_num = n_s_acs[first_index]
    
    # Create Support Space Vectors:
    # Will create 7 support space vectors: -3se, -2se, -1se, 0 , se, 2se, 3se. 
    # Can choose to maximize entropy over a 7 or 3 dimensional support space vector. 
    # Note that support space vectors are PUMA spesific - in other words, each PUMA has thier own support space. 

    ##Save S.E.
    se_p = data2['w_spmu_poor_ses_p']
    nse_p = np.matrix(np.reshape(se_p.to_numpy(), (num_pumas,1)))

    ##Create vecotrs for each support space element: 
    #v1_p = np.multiply(nse_p, -3) # (-3 x s.e.)
    v1_p = np.multiply(nse_p, -10)  # (-10 x s.e.)
    v2_p = np.multiply(nse_p, -2)  # (-2 x s.e.)
    v3_p = np.multiply(nse_p, -1)  # (-1 x s.e.)
    v4_p = np.multiply(nse_p, 0)   # (0 x s.e.)
    v5_p = np.multiply(nse_p, 1)   # (1 x s.e.)
    v6_p = np.multiply(nse_p, 2)   # (2 x s.e.)
    #v7_p = np.multiply(nse_p, 3)  # (3 x s.e.)
    v7_p = np.multiply(nse_p, 10)   # (10 x s.e.)

    # Check to make sure that error bounds do not exceed [0,1]. 
    lower_check = nr_sp_acs + v1_p
    upper_check = nr_sp_acs + v7_p
        #print(lower_check)
        #print(upper_check)
    if ((lower_check > 0).sum() == lower_check.size).astype(np.int) != 1:
        # sys.exit("Error: Lower bound exceeds [0,1]") 
        # Differences between ACS and CPS Estimates are so large for some states that the support space defined above
        # exceeds the rational bounds. If lower support space bound is less than zero, replace it with zero. 
        for n, i in enumerate(lower_check):
            if i < 0:
                lower_check[n] = 0
        

    if ((upper_check > 1).sum() == upper_check.size).astype(np.int) != 1:
        sys.exit("Error: Upper bound exceeds [0,1]")
        
    #Create Matrix of Unkown Probabilities 
    #Creating a `num_pumas` by 7/3 matrix (7/3 because there are 7/3 dimensional support spaces for each PUMA). 
    #The matrix "w" will be the matrix associated with the probabilities of the support space outcomes (for each PUMA i.e. row). 
    #Each row in this matrix will have 7/3 elements: w1_p ... w4_p ... w7_p. 

    # Seven Dimensional Support Space ("Turn on" as needed/wanted.)
        #w_7 = cp.Variable((num_pumas,7),"wd_p")

    # Three Dimensional Support Space 
    w_3 = cp.Variable((num_pumas,3),"wd_p")
    
    # Test that the constraint is correctly specified before using it in the max. ent. problem below. 
    #test_3 = sum( nn_sp_acs[i,0] * (nr_sp_acs[i,0] - (w_3[i,0] * v1_p[i,0]) - (w_3[i,1] * v4_p[i,0]) - ( w_3[i,2] * v7_p[i,0])) for i in range(num_pumas))
    #print(test_3)
    
    # MAXIMUM ENTROPY PROBLEM *******
    objective_3 = cp. Maximize(cp.sum(cp.entr(w_3)))
 
    constraints_3 = []
    constraints_3 += [ w_3[i,j] >= 0 for i in range(num_pumas) for j in range (3) ] # All elemnts (probabilities) in support space Matrix are greater than zero.  
    constraints_3 += [ w_3[i,0] + w_3[i,1] + w_3[i,2] == 1 for i in range(num_pumas) ] # Probabilities spanning each row sum to one. 
    constraints_3 += [ r_s_cps_num == (sum(((nr_sp_acs[i,0] - (w_3[i,0] * v1_p[i,0]) - (w_3[i,1] * v4_p[i,0]) - (w_3[i,2] * v7_p[i,0])) * nn_sp_acs[i,0])  for i in range(num_pumas)) / n_s_acs_num) ]
    constraints_3 += [ (nr_sp_acs[i,0] - (w_3[i,0] * v1_p[i,0]) - (w_3[i,1] * v4_p[i,0]) - ( w_3[i,2] * v7_p[i,0])) >= 0 for i in range(num_pumas) ] #Estimated "true" number of poor cannot be negative. 
    constraints_3 += [ (nr_sp_acs[i,0] - (w_3[i,0] * v1_p[i,0]) - (w_3[i,1] * v4_p[i,0]) - ( w_3[i,2] * v7_p[i,0])) <= 1 for i in range(num_pumas) ] #Estimated "true" SPM PUMA rate must be less than one. 

    prob = cp.Problem(objective_3, constraints_3)
    #prob.solve(solver=cp.SCS)
    prob.solve(warm_start = True, solver=cp.SCS, verbose=False, max_iters=200000) # Elissa's prob.solve command. 

    est_w3 = w_3.value
    #print(est_w3)
    
    #Use the results from the estimation to refine ACS PUMA SPM Estimates:
    #Grab columns from max. entropy result and save them as matricies 
    est_w1_p = np.transpose(np.matrix(column(est_w3, 0)))
    est_w4_p = np.transpose(np.matrix(column(est_w3, 1)))
    est_w7_p = np.transpose(np.matrix(column(est_w3, 2)))

    #Multiply each column by respective support element. 
    w1_v1_p = np.multiply(est_w1_p, v1_p)
    w4_v4_p = np.multiply(est_w4_p, v4_p)
    w7_v7_p = np.multiply(est_w7_p, v7_p)

    #Sum the row element for across each of the three vectors to get approximate error
    error = np.add(np.add(w1_v1_p, w4_v4_p), w7_v7_p)
    #print(error)

    #Subtract error from the original PUMA rates
    est_r_sp_acs = np.subtract(nr_sp_acs, error)
    
    #Check estimates satisfy observed constraint. 
    numerator = sum(np.multiply(est_r_sp_acs[i,0], nn_sp_acs[i,0]) for i in range(num_pumas)) / n_s_acs_num
    #print(numerator) #Resulting state SPM rate from new, estimates PUMA rates 
    #print(r_s_cps_num) #original state SPM rate from CPS ASEC 

    if (round(numerator,4) != round(r_s_cps_num,4)):
        sys.exit("Estimated error does not satisfy observed constraint.")
        
    # Make vector into data frame
    df = pd.DataFrame(data = est_r_sp_acs, columns = ['max_ent_ests'])
    df_ests = df.set_index(np.transpose(index_list)) # Give estimates appropriate index values. 

    #Append with original data frame
    data3 = pd.concat([data2, df_ests], axis=1)

    #Export dataset to a new XLSX doc
    data3.to_excel(r"C:\Users\Danielle Wilson\Dropbox\SPM_ASEC_ACS_project\Python_exports\max_ent_ests_gestfips_" + str(s) + ".xlsx",  sheet_name= 'Gestfips ' + str(s))